# Новый анализ

In [10]:
import pandas as pd

# Загрузка данных
data_path = './data/clean_data.csv'
data = pd.read_csv(data_path)

# Просмотр первых строк данных
print(data.head())

  Специальность   1. Врач должен регулярно тесно общаться с пациентами  \
0    Эндоскопия                                                  4       
1    Эндоскопия                                                  3       
2    Эндоскопия                                                  5       
3    Эндоскопия                                                  5       
4    Эндоскопия                                                  5       

    2. Врач работает со сложными переживаниями пациентов: сообщает плохие новости, улаживает конфликты  \
0                                                  2                                                     
1                                                  3                                                     
2                                                  4                                                     
3                                                  5                                                     
4                        